## Arboles de decisión I
- - -

Los Árboles de Decisión tienen el objetivo de crear un modelo que prediga el valor de la respuesta mediante el aprendizaje de reglas de decisión simples inferidas a partir de las características de los datos.

Los nodos internos de un árbol de decisión representan una serie de reglas de partición sobre las observaciones.

Cada hoja contiene una predicción.

<img src=https://i.ibb.co/qsCfFYd/01.png width="500">

- - -

In [ ]:
import pandas as pd

In [ ]:
padel = pd.read_csv("https://raw.githubusercontent.com/4data-lab/datasets/master/padel.csv")

In [ ]:
padel

- - -
*Contenido del dataset*:

0. outlook (tiempo) = overcast (nublado) / rainy (lluvioso) / sunny (soleado)
1. temperature
2. humidity
3. windy


4. play  
    - yes
    - no
- - -

<img src=https://i.ibb.co/87NL5Bh/02.png width="500">

- - -

In [ ]:
nba = pd.read_csv("https://raw.githubusercontent.com/4data-lab/datasets/master/nba_logreg.csv")

Nuestro dataset contiene estadísticas de los rookies (novatos) de la NBA para predecir si un jugador durará 5 años en la liga.

In [ ]:
nba.head()

In [ ]:
nba.shape

In [ ]:
nba.isnull()

    Usando .any().any() obtendremos un valor de True o False indicando si en todo el dataset hay o no algún missing value.

In [ ]:
nba.isnull().any().any()

    Usando solo un .any() obtendremos un valor de True o False indicando si en alguna columna hay o no algun missing value.

In [ ]:
nba.isnull().any()

In [ ]:
nba.isnull().sum()

- - -

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(nba.isnull(), yticklabels=False, cbar=False)

In [ ]:
nba = nba.dropna()

- - -

In [ ]:
X = nba.drop(["name", "target_5yrs"], axis=1)

In [ ]:
y = nba["target_5yrs"]

In [ ]:
X.head()

In [ ]:
y.value_counts()

- - -

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.30, random_state=1)

- - -

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DTC = DecisionTreeClassifier(random_state=1)

In [ ]:
DTC.fit(X_train, y_train)

- - -

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

In [ ]:
y_train_pred = DTC.predict_proba(X_train)

In [ ]:
print("AUC - Datos de train")
roc_auc_score(y_train, y_train_pred[:, 1])

In [ ]:
print("AUC - Datos de validation")
cross_val_score(DTC, X_train, y_train, cv=5, scoring="roc_auc").mean()

In [ ]:
DTC.tree_.max_depth

- - -

## Overfitting

<img src=https://i.ibb.co/yhxsmrc/03.png width="700">

El sobreajuste es la producción de un análisis que se corresponde demasiado o exactamente con un conjunto particular de datos.

Por lo tanto, puede no ajustarse a datos adicionales o predecir futuras observaciones de manera confiable.

Durante la fase de sobreajuste, el éxito al responder las muestras de entrenamiento sigue incrementándose mientras que su actuación con muestras nuevas va empeorando.
- - -

In [ ]:
train_score = []
test_score = []
max_depth = list(range(1, 10))

In [ ]:
for i in max_depth:
    DTC = DecisionTreeClassifier(random_state=1, max_depth=i)
    DTC.fit(X_train,y_train)
    y_train_pred = DTC.predict_proba(X_train)
    train_score.append(roc_auc_score(y_train, y_train_pred[:, 1]))
    test_score.append(cross_val_score(DTC, X_train, y_train, cv=5, scoring="roc_auc").mean())

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(max_depth, train_score, color="blue")
plt.plot(max_depth, test_score, color="red")

- - -

In [ ]:
DTC = DecisionTreeClassifier(random_state=1, max_depth=3)
DTC.fit(X_train,y_train)

In [ ]:
print("AUC - Datos de validation")
cross_val_score(DTC, X_train, y_train, cv=5, scoring="roc_auc").mean()

In [ ]:
print("AUC - Datos de test")
y_test_pred = DTC.predict_proba(X_test)
roc_auc_score(y_test, y_test_pred[:, 1])

In [ ]:
#Podemos pintar el árbol
from sklearn import tree
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(20,10))
target_names = y.unique()
tree.plot_tree(DTC, feature_names=X.columns, class_names=target_names.astype(str),
                   filled=True, impurity = False);

In [ ]:
# También lo podemos printar en texto
from sklearn.tree import export_text
tree_rules = export_text(DTC, feature_names=X.columns.tolist())
print(tree_rules)

- - -